# Modelo de lenguaje de Latino 40

Vamos a hacer el archivo que necesitamos para el reconocimiento con el HTK.

In [1]:
import pandas as pd
import torch
from NLPUtils import *
import re
import fasttext

%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%autoreload 2

## Modelo de lenguaje frecuentista

Creamos el archivo `lm_l40_bigram_train_test` dentro de la carpeta `lm_files` con el programa *ngram-count* del *SRILM*. Este archivo contiene la información del modelo de lenguaje estimado para un modelo de bigramas con el set de entrenamiento y de test de la base de datos Latino-40.

El archivo `vocab` consta de todas las palabras de la base de datos de test. La instrucción utilizada a continuación cuenta la cantidad de bigramas en un archivo `trainLM2.txt` y con eso estima la probabilidad del modelo de lenguaje. El parámetro `-order 2` indica que se implementa un modelo de bigramas y el parametro `-unkdiscount2` indica que se usa el método de suavizado de Kneser-Ney.

In [2]:
!/usr/local/speechapp/srilm/bin/i686-m64/ngram-count -order 2 -text trainLM2.txt -lm ./lm_files/lm_l40_bigram_train_test -ukndiscount2 -vocab vocab

Calculamos la perplejidad sobre el corpus de test para este modelo de lenguaje.

In [3]:
# Obtenemos el modelo de lenguaje del método frecuentista:
with open('lm_l40_2', 'rb') as file:
    lm_file = file.read().decode('iso-8859-1')
    
def get_log_prob(w1,w2):
    match = re.search(r'([\-]?[\d]+[\.]?[\d]*)\t({} {})'.format(w1,w2),lm_file)
    if match is not None:
        log_prob, _ = match.groups()
        return float(log_prob)
    match_w1 = re.search(r'([\-]?[\d]+[\.]?[\d]*)\t({})\t*(\-?[\d]*[\.]?[\d]*)'.format(w1),lm_file)
    match_w2 = re.search(r'([\-]?[\d]+[\.]?[\d]*)\t({})\t*(\-?[\d]*[\.]?[\d]*)'.format(w2),lm_file)
    return float(match_w1.groups()[0]) + float(match_w2.groups()[2])   
        
# Juntamos el corpus de test en una sola lista:
with open('promptsl40.test','rb') as file:
    test_lines = file.readlines()
    test_lines = [' '.join(re.findall(r'\w+',line.decode('iso-8859-1'))[1:]) for line in test_lines]
    corpus_test = [['<s>'] + line.split(' ') + ['</s>'] for line in test_lines]
    corpus_test = [word for line in corpus_test for word in line]
    
# Perplejidad para un modelo de bigrama:
corpus_len = len(corpus_test)
log_p = [get_log_prob(corpus_test[idx-1],corpus_test[idx]) for idx in range(1,corpus_len)]
log_p.insert(0,float(re.search(r'([\-]?[\d]+[\.]?[\d]*)\t({})\t*(\-?[\d]*[\.]?[\d]*)'.format(corpus_test[0]),lm_file).groups()[0]))
print('El logaritmo de la perplejidad para el corpus de test es: {}'.format(sum(log_p)/corpus_len))

El logaritmo de la perplejidad para el corpus de test es: -1.3650002757790303


Mostrar resultados del reconocimiento

## Modelo de lenguaje neuronal

Vamos a entrenar un modelo de lenguaje con una red neuronal a partir de las frases de entrenamiento y de test, las cuales se encuentran en el archivo `trainLM2.txt`.

In [10]:
# Obtenemos los bigramas que tenemos que entrenar:
with open('promptsl40.test','rb') as file:
    test_lines = file.readlines()
    test_lines = [' '.join(re.findall(r'\w+',line.decode('iso-8859-1'))[1:]) for line in test_lines]
corpus_test = [['<s>'] + line.split(' ') + ['</s>'] for line in test_lines]
corpus_test = [word for line in corpus_test for word in line]
corpus_test_len = len(corpus_test)
bigrams = sorted(list(set(['{} {}'.format(corpus_test[t-1],corpus_test[t]) for t in range(1,corpus_test_len)] )))
unigrams = sorted(list(set(corpus_test)))

# Obtenemos el corpus de entrenamiento:
with open('trainLM2.txt', 'rb') as file:
    lines = file.readlines()
    corpus = [['<s>'] + line.decode('iso-8859-1').split(' ')[:-1] + ['</s>'] for line in lines]
    
corpus = [[token for doc in corpus for token in doc]]

In [5]:
# Entrenamos:

window_size = 8           # Tamaño de la ventana del contexto.
cutoff_freq = 0           # Palabras con una frecuencia menor o igual a cutoff_freq son excluídas del vocabulario.
batch_size = 512          # Tamaño del batch.

model = 'CBOW'            # Método de entrenamiento.
embedding_dim = 200       # Dimensión del espacio de los word vectors.
device = 'cuda:1'         # Dispositivo sobre el cual se entrena. 
state_dict = None         # Parámetros pre-entrenados.
paralelize = False        # Flag para decirle al programa que use las 2 gpus

epochs = 100              # Cantidad de epochs
learning_rate = 5e-4      # Tasa de aprendizaje
sample_loss_every = 10    # Calcular la loss cada este número
algorithm = 'Adam'        # Algoritmo de optimización

trainer = Word2vecTrainer(corpus,cutoff_freq=cutoff_freq,window_size=window_size,batch_size=batch_size)
trainer.InitModel(model=model, state_dict=state_dict, device=device, paralelize=paralelize, embedding_dim=embedding_dim)
trainer.Train(algorithm=algorithm, epochs=epochs, sample_loss_every=sample_loss_every, lr=learning_rate)

Word2vec trainer created:
Window size: 8
Number of samples: 48612
Vocabulary Size: 5924
Number of batches: 95
Number of samples per batch: 512

Dispositivo seleccionado: cuda:1
Dimensión del espacio de los embeddings: 200
Starting training...
Optimization method: Adam
Learning Rate: 0.0005
Number of epochs: 100
Running on device (cuda:1)

Epoch: 1, Batch number: 0, Loss: 4446.98486328125
Epoch: 1, Batch number: 10, Loss: 4385.72216796875
Epoch: 1, Batch number: 20, Loss: 4316.03369140625
Epoch: 1, Batch number: 30, Loss: 4267.333984375
Epoch: 1, Batch number: 40, Loss: 4206.4677734375
Epoch: 1, Batch number: 50, Loss: 4144.2373046875
Epoch: 1, Batch number: 60, Loss: 4102.51416015625
Epoch: 1, Batch number: 70, Loss: 4000.6240234375
Epoch: 1, Batch number: 80, Loss: 3973.108154296875
Epoch: 1, Batch number: 90, Loss: 3927.130615234375
Epoch: 2, Batch number: 5, Loss: 3808.236083984375
Epoch: 2, Batch number: 15, Loss: 3734.701904296875
Epoch: 2, Batch number: 25, Loss: 3672.93530273437

Epoch: 17, Batch number: 40, Loss: 2293.56591796875
Epoch: 17, Batch number: 50, Loss: 2275.305419921875
Epoch: 17, Batch number: 60, Loss: 2247.67431640625
Epoch: 17, Batch number: 70, Loss: 2247.184814453125
Epoch: 17, Batch number: 80, Loss: 2239.06103515625
Epoch: 17, Batch number: 90, Loss: 2185.040771484375
Epoch: 18, Batch number: 5, Loss: 2288.3623046875
Epoch: 18, Batch number: 15, Loss: 2212.8515625
Epoch: 18, Batch number: 25, Loss: 2234.97802734375
Epoch: 18, Batch number: 35, Loss: 2204.158203125
Epoch: 18, Batch number: 45, Loss: 2224.71142578125
Epoch: 18, Batch number: 55, Loss: 2118.0380859375
Epoch: 18, Batch number: 65, Loss: 2247.163330078125
Epoch: 18, Batch number: 75, Loss: 2224.26513671875
Epoch: 18, Batch number: 85, Loss: 2211.038330078125
Epoch: 19, Batch number: 0, Loss: 2164.0732421875
Epoch: 19, Batch number: 10, Loss: 2207.3828125
Epoch: 19, Batch number: 20, Loss: 2152.52490234375
Epoch: 19, Batch number: 30, Loss: 2198.706298828125
Epoch: 19, Batch numb

Guardamos el modelo de lenguaje en el archivo `lm_l40_word_vectors` y calculamos la perplejidad:

In [26]:
def compute_probs(trainer,bigrams):
    vocab = trainer.dataloader.dataset.vocabulary
    forward = lambda x: trainer.model.out(trainer.model.emb(x))
    device = trainer.device
    log_probs_bigram = []
    log_probs_unigram = []
    for bigram in bigrams:
        w1, w2 = bigram.split(' ')
        score = forward(torch.tensor(vocab[w1],device=device))
        log_prob = max((score[vocab[w2]] - torch.logsumexp(score,dim=0)).item(),-99)
        log_probs_bigram.append(log_prob)
    for unigram in unigrams:
        score = forward(torch.tensor(vocab[unigram],device=device))
        log_prob = max((score[vocab[unigram]] - torch.logsumexp(score,dim=0)).item(),-99)
        log_probs_unigram.append(log_prob)
    bigrams_with_probs = ['{:.6f}\t{}\n'.format(log_prob,bigram) for log_prob, bigram in zip(log_probs_bigram, bigrams)]
    unigrams_with_probs = ['{:.6f}\t{}\t0\n'.format(log_prob,unigram) for log_prob, unigram in zip(log_probs_unigram, unigrams)]
    text = r"""
\data\

ngram 1={}
ngram 2={}

\1-grams:
{}

\2-grams:
{}

\end\
""".format(len(unigrams),len(bigrams),''.join(unigrams_with_probs),''.join(bigrams_with_probs))
    return text
    
with open('./lm_files/lm_l40_word_vectors', 'wb') as file:
    text = compute_probs(trainer,bigrams)
    file.write(text.encode('iso-8859-1'))
print(text)


\data\

ngram 1=2756
ngram 2=9568

\1-grams:
-99.000000	</s>	0
-99.000000	<s>	0
-31.683832	a	0
-12.044575	abajo	0
-18.724909	abandonada	0
-16.893559	abandonado	0
-8.784182	abarcan	0
-14.882177	abastecimiento	0
-16.827162	abatir	0
-11.647960	abierta	0
-16.938824	abiertas	0
-14.339309	abierto	0
-14.410190	abra	0
-10.536858	abrigamos	0
-12.185606	abril	0
-13.437931	abrió	0
-12.892111	abstracto	0
-10.567724	abundan	0
-12.115994	aburrida	0
-20.159836	acaba	0
-12.435965	acaben	0
-10.619644	acaso	0
-14.578949	acatamiento	0
-15.336414	acceder	0
-16.298889	acceso	0
-13.046727	accidente	0
-17.460117	acciones	0
-17.963173	acción	0
-19.371288	acelerar	0
-12.926479	acento	0
-12.997189	aceptable	0
-19.035316	aceptar	0
-18.346920	aceptarla	0
-12.045515	acepto	0
-14.937613	aceptó	0
-8.648340	aclara	0
-15.262013	aclararlo	0
-13.934893	acomete	0
-22.307552	acontecimiento	0
-8.970358	acopla	0
-16.037809	acordaron	0
-13.058916	acordó	0
-9.167282	acortarán	0
-13.596361	acosta	0
-9.842801	actitud	0
-11.951

In [23]:
'abajo </s>' in bigrams

False

In [13]:
# Perplejidad para un modelo de bigrama:
def get_log_prob(w1,w2):
    match = re.search(r'([\-]?[\d]+[\.]?[\d]*)\t({} {})'.format(w1,w2),text)
    if match is not None:
        log_prob, _ = match.groups()
        return float(log_prob)
    match_w1 = re.search(r'([\-]?[\d]+[\.]?[\d]*)\t({})\t*(\-?[\d]*[\.]?[\d]*)'.format(w1),text)
    match_w2 = re.search(r'([\-]?[\d]+[\.]?[\d]*)\t({})\t*(\-?[\d]*[\.]?[\d]*)'.format(w2),text)
    return float(match_w1.groups()[0]) + float(match_w2.groups()[2])  

corpus_len = len(corpus_test)
log_p = [get_log_prob(corpus_test[idx-1],corpus_test[idx]) for idx in range(1,corpus_len)]
log_p.insert(0,float(re.search(r'([\-]?[\d]+[\.]?[\d]*)\t({})\t*(\-?[\d]*[\.]?[\d]*)'.format(corpus_test[0]),text).groups()[0]))
print('El logaritmo de la perplejidad para el corpus de test es: {}'.format(sum(log_p)/corpus_len))

El logaritmo de la perplejidad para el corpus de test es: -14.52675450222583
